In [4]:
import pandas as pd
import pyopenms as oms
import json 
import time
import os
from mass_trace import run_mass_trace_detection
from elution_peak import run_elution_peak_detection
from feature_map import run_feature_mapping
from align_chromatograms import align_featureXML_files
from link_features import link_features
from hmdb_indexing import parse_hmdb_to_dataframe_streaming, consensus_to_feature_dicts, load_kegg_compounds_csv
from batch_correction import normalize_feature_maps
from quality_control import run_qc_advanced
from final_report import run_final_report
from adduct_isotope_annotation import annotate_adducts_isotopes

In [17]:
import requests
from dotenv import load_dotenv
import numpy as np
import subprocess 
load_dotenv('../metabolomics_microservice/.env')
METABOLOMICS_SAVE_PATH = os.getenv("METABOLOMICS_SAVE_PATH")
TOOLS_PATH = os.getenv("TOOLS_PATH")
THERMO_FILE_PARSER_PATH = os.path.join("/home/fdotta/dev/omnis_microservices_platform/metabolomics_microservice/","thermofisher", "ThermoRawFileParser.exe" )

In [20]:


def convert_raw_to_mzml(raw_file_path):
    # Construct the command to convert the raw file to mzML
    command = [
        "mono", THERMO_FILE_PARSER_PATH,
        "-i", raw_file_path,
        "-o", "/media/datastorage/it_cast/metabolomica/test",
        "-f", "1"  # Format 1 corresponds to mzML
    ]

    # Run the command
    subprocess.run(command, check=True)

    # Return the path of the converted mzML file
    mzml_file_path = os.path.join(raw_file_path.replace('.raw', '.mzML'))
    return mzml_file_path

In [23]:
mzml_file_1 = convert_raw_to_mzml("/media/datastorage/it_cast/metabolomica/test/CC1.raw")
mzml_file_2 = convert_raw_to_mzml("/media/datastorage/it_cast/metabolomica/test/CC2.raw")
mzml_file_3 = convert_raw_to_mzml("/media/datastorage/it_cast/metabolomica/test/CC3.raw")

2025-09-09 18:39:31 INFO Started parsing /media/datastorage/it_cast/metabolomica/test/CC1.raw
2025-09-09 18:39:32 INFO Processing 11919 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2025-09-09 18:40:04 INFO Finished parsing /media/datastorage/it_cast/metabolomica/test/CC1.raw
2025-09-09 18:40:04 INFO Processing completed 0 errors, 0 warnings
2025-09-09 18:40:04 INFO Started parsing /media/datastorage/it_cast/metabolomica/test/CC2.raw
2025-09-09 18:40:06 INFO Processing 11919 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2025-09-09 18:40:39 INFO Finished parsing /media/datastorage/it_cast/metabolomica/test/CC2.raw
2025-09-09 18:40:39 INFO Processing completed 0 errors, 0 warnings
2025-09-09 18:40:39 INFO Started parsing /media/datastorage/it_cast/metabolomica/test/CC3.raw
2025-09-09 18:40:41 INFO Processing 11914 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2025-09-09 18:41:11 INFO Finished parsing /media/datastorage/it_cast/metabolomica/test/CC3.raw
2025-09-09 18:41:

In [2]:
### FUNCTION TO GET THE STEPS OF THE JSON PIPELINE
def get_pipeline_steps(pipeline_json):
    mzml_file = []
    steps = pipeline_json.get('pipeline', {}).get('steps', [])
    for step in steps:
        if step.get('name') == 'select_mzml_file':
            mzml_files = step.get('parameters', {}).get('mzml_files', [])
    return steps, mzml_files

In [3]:
file = "/media/datastorage/it_cast/metabolomica/test/extracted/AMI-001.mzML"
exp = oms.MSExperiment()
oms.MzMLFile().load(file,exp)
print(f"Loaded {exp.size()} spectra from {file}")

Loaded 0 spectra from /media/datastorage/it_cast/metabolomica/test/extracted/AMI-001.mzML
